In [55]:
import pickle
import numpy
with open("aa_sim_array.pickle", 'rb') as f:
    aa_mat = pickle.load(f)
with open("pos_sim_array.pickle", 'rb') as f:
    str_mat = pickle.load(f)
with open("pos_pdb_ids.pickle", 'rb') as f:
    ids = pickle.load(f)
with open("pos_labels.pickle", 'rb') as f:
    labels = pickle.load(f)

In [56]:
str_mat #Levenshtein distances - the bigger the less similar

array([[0.        , 0.29230769, 0.29007634, ..., 1.55932203, 1.57627119,
        1.29166667],
       [0.        , 0.        , 0.06923077, ..., 1.38983051, 1.40677966,
        1.13888889],
       [0.        , 0.        , 0.        , ..., 1.37288136, 1.38983051,
        1.13888889],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.20338983,
        0.40677966],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.44067797],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [57]:
aa_mat #Levenshtein distances - the bigger the less similar

array([[0.        , 0.71942446, 0.71942446, ..., 2.        , 2.        ,
        1.61842105],
       [0.        , 0.        , 0.        , ..., 1.80645161, 1.80645161,
        1.48684211],
       [0.        , 0.        , 0.        , ..., 1.80645161, 1.80645161,
        1.48684211],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.88709677],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.88709677],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [62]:
#Check
aa_mat[263,265]

0.5231607629427792

In [63]:
np.where(np.array(ids) == '3OJY:A')

(array([265], dtype=int64),)

In [64]:
np.where(np.array(ids) == '2RD7:A')

(array([263], dtype=int64),)

In [65]:
#interested in big aa difference, but small str difference -> aa_mat / str_mat -> the bigger the better (more interesting)

str_mat = str_mat + 0.00001
res = aa_mat / str_mat

In [66]:
import numpy as np
res = np.array(res)
np.max(res)

2127.659574468085

In [67]:
import bottleneck as bn

def top_n_indexes(arr, n):
    idx = bn.argpartition(arr, arr.size-n, axis=None)[-n:]
    width = arr.shape[1]
    return [divmod(i, width) for i in idx]

In [68]:
idx = top_n_indexes(res, 3000)
idx.sort(key = lambda tup: tup[0])

In [69]:
temp = np.partition(-res.flatten(), 5)
-temp[:5]

array([2127.65957447, 2127.65957447,  513.69863014,   11.73043078,
         10.94649216])

In [70]:
#For twilight zone, we're only interested if aa distance is bigger than 0.75
#Note that the residues (e.g. 34:260) are not taken into account and we select the whole amino acid / structure sequence
count = 0
important_ids = []
important_labels = []
indices = []
used_idx= []
for i in idx:
    if (aa_mat[i[0], i[1]] > 0.7) and (str_mat[i[0], i[1]] < 0.25):
        important_ids.append(ids[i[0]])
        important_ids.append(ids[i[1]])
        important_labels.append(labels[i[0]])
        important_labels.append(labels[i[1]])
        indices.append(i[0])
        indices.append(i[1])
        used_idx.append((i[0], i[1]))
        count = count + 1
        print('"' + ids[i[0]] + '"' + ' paired with ' + '"' + ids[i[1]] + '"' + ' yields ' + str(res[i[0], i[1]]) + ' with aa distance ' + str(aa_mat[i[0], i[1]]) + ' and labels ' + labels[i[0]] + ' and ' + labels[i[1]])

"3RON:A" paired with "2RCI:A" yields 4.115695433763071 with aa distance 0.7647058823529411 and labels pos_Bac_thur_toxin.txt and pos_Bac_thur_toxin.txt
"2D42:B" paired with "4RHZ:A" yields 4.456516751916893 with aa distance 0.8152610441767069 and labels pos_ETX_MTX2.txt and pos_ETX_MTX2.txt
"4Q7G:A" paired with "3ROH:A" yields 3.1131142388059203 with aa distance 0.7253086419753086 and labels pos_Leukocidin.txt and pos_Leukocidin.txt
"4Q7G:A" paired with "5K59:C" yields 3.186066709616501 with aa distance 0.7241379310344828 and labels pos_Leukocidin.txt and pos_Leukocidin.txt
"4Q7G:A" paired with "4IYA:A" yields 3.402409312314751 with aa distance 0.7534246575342466 and labels pos_Leukocidin.txt and pos_Leukocidin.txt
"4Q7G:A" paired with "4IYC:B" yields 3.442464936035723 with aa distance 0.75 and labels pos_Leukocidin.txt and pos_Leukocidin.txt
"4Q7G:A" paired with "4IYT:A" yields 3.181683201318732 with aa distance 0.75 and labels pos_Leukocidin.txt and pos_Leukocidin.txt
"4Q7G:A" paired

In [71]:
count

99

In [72]:
np.unique(np.array(important_ids))

array(['1LKF:A', '1PVL:A', '1T5R:A', '2D42:B', '2LKF:A', '2QK7:A',
       '2QK7:B', '2RCI:A', '2YGT:A', '3ANZ:H', '3B07:A', '3B07:F',
       '3HVN:A', '3LKF:A', '3M3R:D', '3ROH:A', '3RON:A', '3WZ3:A',
       '3WZ4:E', '3WZ5:B', '4CDB:A', '4HSC:X', '4I0N:A', '4IYA:A',
       '4IYC:B', '4IYT:A', '4IZL:D', '4J0O:B', '4P1X:G', '4P1X:H',
       '4P1Y:A', '4P1Y:H', '4P24:C', '4Q7G:A', '4QQA:A', '4QQQ:A',
       '4RHZ:A', '4U6V:B', '4YHD:C', '5AOD:A', '5AOE:B', '5AOF:A',
       '5CNL:B', '5CR6:D', '5IMY:A', '5K59:B', '5K59:C'], dtype='<U6')

In [73]:
len(np.unique(np.array(important_ids)))

47

In [74]:
np.unique(np.array(important_labels))

array(['pos_Bac_thur_toxin.txt', 'pos_ETX_MTX2.txt', 'pos_Leukocidin.txt',
       'pos_T4BSS_DotI_IcmL.txt', 'pos_Thiol_cytolysin.txt'], dtype='<U23')

In [75]:
labels_from_indices = [labels[i] for i in np.unique(indices)]

In [76]:
from collections import Counter
print(Counter(labels_from_indices))

Counter({'pos_Leukocidin.txt': 29, 'pos_Thiol_cytolysin.txt': 10, 'pos_T4BSS_DotI_IcmL.txt': 4, 'pos_Bac_thur_toxin.txt': 2, 'pos_ETX_MTX2.txt': 2})


In [77]:
total = aa_mat.shape[0]

In [78]:
for i in range(total-1, 0, -1):
    for j in range(i-1, -1, -1):
        aa_mat[i,j] = aa_mat[j,i]

In [79]:
for i in range(total-1, 0, -1):
    for j in range(i-1, -1, -1):
        str_mat[i,j] = str_mat[j,i]

In [80]:
aa_mat[:5,:5]

array([[0.        , 0.71942446, 0.71942446, 0.7173913 , 0.70289855],
       [0.71942446, 0.        , 0.        , 0.01449275, 0.07246377],
       [0.71942446, 0.        , 0.        , 0.01449275, 0.07246377],
       [0.7173913 , 0.01449275, 0.01449275, 0.        , 0.07246377],
       [0.70289855, 0.07246377, 0.07246377, 0.07246377, 0.        ]])

In [81]:
#Sequence cluster
import umap
import umap.plot
import pandas as pd

mapper = umap.UMAP().fit(aa_mat)
hover_data = pd.DataFrame({'index':np.arange(total),
                           'label':labels})

hover_data['item'] = hover_data.label

C:\Users\sarva\Anaconda3\lib\site-packages\sklearn\manifold\_spectral_embedding.py:236: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  warnings.warn("Graph is not fully connected, spectral embedding"


In [82]:
from bokeh.plotting import show, save, output_notebook, output_file

p = umap.plot.interactive(mapper, labels=labels, hover_data=hover_data,point_size=20)
output_notebook()
show(p)

Loading BokehJS ...

In [83]:
#Structure cluster
mapper = umap.UMAP().fit(str_mat)
hover_data = pd.DataFrame({'index':np.arange(total),
                           'label':labels})

hover_data['item'] = hover_data.label

In [84]:
from bokeh.plotting import show, save, output_notebook, output_file

p = umap.plot.interactive(mapper, labels=labels, hover_data=hover_data,point_size=20)
output_notebook()
show(p)

Loading BokehJS ...

In [85]:
#Number of islands problem - Leetcode 200 

def numIslands(grid):        
    count = 0
    for i in range(grid.shape[0]):
        for j in range(grid.shape[1]):
            if grid[i,j] == 1:
                dfs(grid, i, j)
                count += 1
    return count

def dfs(grid, i, j):
    if i<0 or j<0 or i>=len(grid) or j>=len(grid[0]) or grid[i,j] != 1:
        return
    grid[i][j] = -1
    dfs(grid, i+1, j)
    dfs(grid, i-1, j)
    dfs(grid, i, j+1)
    dfs(grid, i, j-1)

In [86]:
grid = np.zeros((total, total), dtype='int')

for i in used_idx:
    grid[i[0],i[1]] = 1

In [87]:
numIslands(grid)

40

In [88]:
grid2 = np.array([[1,1,0,0,0],[1,1,0,0,0],[0,0,1,0,0],[0,0,0,1,1]])

In [89]:
#Sanity check
numIslands(grid2)

3